<a href="https://colab.research.google.com/github/donbcolab/AIE3/blob/main/paligemma_cnmc_finetune_v10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paligemma Fine Tuning using CNMC dataset v10

### Setting Up

In [1]:
# Constants
base_model_name = "google/paligemma-3b-pt-224"
adapter_version = "paligemma-cnmc-ft"
adapter_model_name = f"dwb2023/{adapter_version}"
max_seq_length = 128
output_dir = adapter_version

In [2]:
!pip install -q -U git+https://github.com/huggingface/transformers.git bitsandbytes datasets accelerate peft hf_transfer

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import PaliGemmaForConditionalGeneration, BitsAndBytesConfig, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig
from transformers import AutoProcessor
from datasets import load_dataset, DatasetDict, Image
from google.colab import userdata
import os

# Constants
base_model_name = "google/paligemma-3b-pt-224"
adapter_version = "paligemma-cnmc-ft"
adapter_model_name = f"dwb2023/{adapter_version}"
max_seq_length = 128
output_dir = adapter_version

# HF Token
HF_TOKEN = userdata.get('HF_TOKEN')
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [ ]:
# Load Dataset and Processor
ds = load_dataset("dwb2023/cnmc-leukemia-2019", split="train")
processor = AutoProcessor.from_pretrained(base_model_name)

# Filter records to only include those from fold 0
ds_fold_0 = ds.filter(lambda example: example['fold'] == 0)

# Define the percentage you want to retrieve (e.g., 10%)
percentage = 0.10

# Use train_test_split to get the subset
cnmc_ds = ds_fold_0.train_test_split(test_size=percentage)["test"]

# Columns to remove
cols_remove = ["subject_id", "image_number", "fold", "original_image_name", "relative_file_path"]
cnmc_ds = cnmc_ds.remove_columns(cols_remove)

# Create train-test split
train_ds = cnmc_ds.train_test_split(test_size=0.2)

# Create test-val split
test_val_ds = train_ds["test"].train_test_split(test_size=0.5)

cnmc_ds_dict = DatasetDict({
    "train": train_ds["train"],
    "test": test_val_ds["test"],
    "validation": test_val_ds["train"]
})

# Cast image column AFTER splitting the dataset
ds_train = cnmc_ds_dict["train"].cast_column("image", Image(decode=True))
ds_eval = cnmc_ds_dict["validation"].cast_column("image", Image(decode=True))

In [ ]:
# Collate Function (optimized)
device = "cuda"

def collate_fn(examples):
    texts = ["Are these cells healthy or cancerous?" for _ in range(len(examples))]
    labels = [example['label'] for example in examples]

    # Image conversion on CPU
    images = [example["image"].convert("RGB") for example in examples]

    tokens = processor(text=texts, images=images, suffix=labels,
                       return_tensors="pt", padding="longest", max_length=max_seq_length)

    # Convert to bfloat16 before moving to GPU
    tokens = tokens.to(torch.bfloat16).to(device)
    return tokens

In [ ]:
# Model Loading and Quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16  # Use bfloat16 to save memory
)

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

model = PaliGemmaForConditionalGeneration.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map={"": 0}  # Load directly to the specified device
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Ensure all parameters require gradients
for param in model.parameters():
    param.requires_grad = True

# Disable cache for gradient checkpointing
model.config.use_cache = False

# Enable gradient checkpointing for memory optimization
model.gradient_checkpointing_enable()

In [3]:
# Training Arguments

args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=2,
    remove_unused_columns=False,
    per_device_train_batch_size=4,  # Reduced batch size
    gradient_accumulation_steps=4,  # Adjusted gradient accumulation
    warmup_steps=2,
    learning_rate=2e-5,
    weight_decay=1e-6,
    adam_beta2=0.999,
    logging_steps=100,
    optim="adamw_hf",
    save_strategy="steps",
    save_steps=1000,
    push_to_hub=True,
    save_total_limit=1,
    bf16=True,
    report_to=["tensorboard"],
    dataloader_pin_memory=False
)

NameError: name 'TrainingArguments' is not defined

In [ ]:
# Clear CUDA cache to free up memory
torch.cuda.empty_cache()

In [ ]:
# Trainer setup and training
trainer = Trainer(
    model=model,
    train_dataset=ds_train,
    eval_dataset=ds_eval,
    data_collator=collate_fn,
    args=args
)

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub(f"dwb2023/{output_dir}")